## Marla Goodman HW - M10

#### Part I.

In [42]:
####
#Author: marla goodman 
#version 1.0
#references: 
#https://www.programiz.com/python-programming/working-csv-files
#https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_bucket
#https://realpython.com/python-boto3-aws-s3/
#CLI aws s3api create-bucket --bucket my-bucket-name --region us-west-2 --create-bucket-configuration LocationConstraint=us-west-2
#https://robertorocha.info/setting-up-a-selenium-web-scraper-on-aws-lambda-with-python/ 
##

###Load libraries
import awscli
import selenium
import boto3
import pandas as pd
import time

from selenium import webdriver

In [43]:
####SCRAPE THE WEBSITE######
###call the webdriver
browser = webdriver.Chrome("/Users/marleegoodman/Downloads/chromedriver")

#enter the url path that needs to be accessed by webdriver
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

#identify xpath of location to select element
inputElement = browser.find_element_by_xpath("/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]")
inputElement.send_keys('0')
inputElement1 = browser.find_element_by_xpath("/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]").click()

#identify the table to scrape
table = browser.find_element_by_css_selector('table.Bordered')

print(table)

<selenium.webdriver.remote.webelement.WebElement (session="975d7ca25283a13e25b24af10e22f65e", element="b176b7e8-e5fe-48ed-9f24-db331509a3eb")>


In [44]:
#####CREATE DATE FRAME#####
#create empty dataframe
df =[]

#loop through dataframe to export table
for row in table.find_elements_by_css_selector('tr'):
      cols = df.append([cell.text for cell in row.find_elements_by_css_selector('td')])


#update dataframe with header 
df = pd.DataFrame(df, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])

#delete first row - row 0 - of 'Nones'
df1=df.drop([0])

display(df1) 

#let's have a look at the data before creating the CSV file and loading it into s3 

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
1,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
2,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
3,(ASMA) American Syrian Multicultural Associati...,42-84-63,273130182,NFP,BROOKLYN,NY
4,#HicksStrong Inc.,48-10-48,842612081,NFP,CLIFTON PARK,NY
5,#WalkAway Foundation,47-15-80,832820906,NFP,CARLSBAD,CA
6,04/11 10:17 PM test,47-13-95,206256427,NFP,ALBANY,NY
7,1/20/21 Action Fund,46-99-13,832210730,NFP,SAN FRANCISCO,CA
8,"10/40 Connections, Inc.",45-70-15,621825230,NFP,HIXSON,TN
9,"1000 Feet Project, Inc",45-00-14,473820859,NFP,NEW YORK,NY
10,1000 Islands Hose Haulers,45-38-38,454570241,NFP,CARTHAGE,NY


In [45]:
###LOAD THE FILE INTO S3####
# prepare csv file name   
pathname = 's3:/module10bucket/'
filename= 'charities_bureau_scrape_' #name of your group
datetime = time.strftime("%Y%m%d%H%M%S") #timestamp
filenames3 = "%s%s%s.csv"%(pathname,filename,datetime) #name of the filepath and csv file

#load file into s3. Pandas actually leverages boto to connect to s3 and can push the file directly into an s3 bucket
df1.to_csv(filenames3, header=True, line_terminator='\n') 

#print success message
print("Successfull uploaded file to location:"+str(filenames3))

Successfull uploaded file to location:s3:/module10bucket/charities_bureau_scrape_20210418180522.csv


#### Part II.

In [46]:
#select the pagelinks by class name
pages = browser.find_elements_by_xpath("//a[starts-with(@title,'Go to page')]")

# save all page links into page_links list
page_links = [] 
for page in pages:
    
    print(page.get_attribute("href"))

    page_links.append(page.get_attribute("href"))

https://www.charitiesnys.com/RegistrySearch/search_charities_action.jsp?orgName=&d-49653-p=2&city=&searchType=contains&reg1=&project=Charities&reg3=&reg2=&ein=0-&orgId=&num1=0&state=none&regType=ALL&num2=
https://www.charitiesnys.com/RegistrySearch/search_charities_action.jsp?orgName=&d-49653-p=3&city=&searchType=contains&reg1=&project=Charities&reg3=&reg2=&ein=0-&orgId=&num1=0&state=none&regType=ALL&num2=
https://www.charitiesnys.com/RegistrySearch/search_charities_action.jsp?orgName=&d-49653-p=4&city=&searchType=contains&reg1=&project=Charities&reg3=&reg2=&ein=0-&orgId=&num1=0&state=none&regType=ALL&num2=
https://www.charitiesnys.com/RegistrySearch/search_charities_action.jsp?orgName=&d-49653-p=5&city=&searchType=contains&reg1=&project=Charities&reg3=&reg2=&ein=0-&orgId=&num1=0&state=none&regType=ALL&num2=
https://www.charitiesnys.com/RegistrySearch/search_charities_action.jsp?orgName=&d-49653-p=6&city=&searchType=contains&reg1=&project=Charities&reg3=&reg2=&ein=0-&orgId=&num1=0&stat

In [47]:
#loop through page_links, open each link, scrape the table, and put the table into a new dataframe 

df2 = []
df2.append(df)

# loop the page_links
for page_link in page_links:
    
    # open link
    print(page_link)
    page_tab = browser.get(page_link)
    
    # identify the table to scrape
    table = browser.find_element_by_css_selector('table.Bordered')
    
    # create empty dataframe
    page_df =[]
    
    # only select td list in the tbody
    tbody = table.find_element_by_css_selector('tbody')

    #loop through dataframe to export table
    for row in tbody.find_elements_by_css_selector('tr'):
          cols = page_df.append([cell.text for cell in row.find_elements_by_css_selector('td')])


    # update dataframe with header 
    page_df = pd.DataFrame(page_df, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])
    # display(page_df) #let's have a look at the data before creating the CSV file and loading it into s3
    
    df_list.append(page_df)
    

# concat all df2 to one dataframe
df_comp = pd.concat(df2)
df_comp

https://www.charitiesnys.com/RegistrySearch/search_charities_action.jsp?orgName=&d-49653-p=2&city=&searchType=contains&reg1=&project=Charities&reg3=&reg2=&ein=0-&orgId=&num1=0&state=none&regType=ALL&num2=
https://www.charitiesnys.com/RegistrySearch/search_charities_action.jsp?orgName=&d-49653-p=3&city=&searchType=contains&reg1=&project=Charities&reg3=&reg2=&ein=0-&orgId=&num1=0&state=none&regType=ALL&num2=
https://www.charitiesnys.com/RegistrySearch/search_charities_action.jsp?orgName=&d-49653-p=4&city=&searchType=contains&reg1=&project=Charities&reg3=&reg2=&ein=0-&orgId=&num1=0&state=none&regType=ALL&num2=
https://www.charitiesnys.com/RegistrySearch/search_charities_action.jsp?orgName=&d-49653-p=5&city=&searchType=contains&reg1=&project=Charities&reg3=&reg2=&ein=0-&orgId=&num1=0&state=none&regType=ALL&num2=
https://www.charitiesnys.com/RegistrySearch/search_charities_action.jsp?orgName=&d-49653-p=6&city=&searchType=contains&reg1=&project=Charities&reg3=&reg2=&ein=0-&orgId=&num1=0&stat

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,None,None,None,None,None,None
1,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
2,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
3,(ASMA) American Syrian Multicultural Associati...,42-84-63,273130182,NFP,BROOKLYN,NY
4,#HicksStrong Inc.,48-10-48,842612081,NFP,CLIFTON PARK,NY
5,#WalkAway Foundation,47-15-80,832820906,NFP,CARLSBAD,CA
6,04/11 10:17 PM test,47-13-95,206256427,NFP,ALBANY,NY
7,1/20/21 Action Fund,46-99-13,832210730,NFP,SAN FRANCISCO,CA
8,"10/40 Connections, Inc.",45-70-15,621825230,NFP,HIXSON,TN
9,"1000 Feet Project, Inc",45-00-14,473820859,NFP,NEW YORK,NY


In [48]:
###LOAD THE FILE INTO S3####
# prepare csv file name   
pathname = 's3:/module10bucket/'
filename= 'charities_bureau_scrape_complete' #name of your group
datetime = time.strftime("%Y%m%d%H%M%S") #timestamp
filenames3 = "%s%s%s.csv"%(pathname,filename,datetime) #name of the filepath and csv file

#load file into s3. Pandas actually leverages boto to connect to s3 and can push the file directly into an s3 bucket
df_comp.to_csv(filenames3, header=True, line_terminator='\n') 

#print success message
print("Successfull uploaded file to location:"+str(filenames3))

Successfull uploaded file to location:s3:/module10bucket/charities_bureau_scrape_complete20210418180640.csv
